In [1]:
%reload_ext autoreload
%autoreload 2

import torch
print(torch.__version__)
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
torch.cuda.is_available()

import json
from pathlib import Path
from typing import Iterator
import time

import datasets
import matplotlib.pyplot as plt
from tokenizers import BertWordPieceTokenizer, Regex, normalizers
from tqdm import tqdm
from transformers import (
    BertConfig,
    BertForMaskedLM,
    BertTokenizerFast,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)
import math
from functools import reduce
from collections import namedtuple
from torch import nn
import torch.nn.functional as F
# from reformer_pytorch import ReformerLM
# from x_transformers import TransformerWrapper, Decoder,Encoder
from torch.utils.data import DataLoader
import math 
from accelerate import Accelerator
import torch.nn.functional as F
# from einops import rearrange, pack, unpack


2.0.1+cu117


/home/miu/miniconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_training='tokemonster'#'bert' , 'tokemonster
# LIMIT_DATASET = 2016 * 4  # keep small for development, set to None for full dataset
#(3090) Para 128 tokens(Bert) e 32000 vocab: 1000 = 6 seg, 10000 = 1min, 100000 = 10min, 1m = 100min, 10m = 16h, 100m = 160h ou 6.6 dias
#(3090) Para 84 tokens(tokenmonster salva 35%) e 24000 vocab(Bert): 1000 = 4.3s, 10000 = 43s, 100000 = 7.1m, 1m = 71min, 10m = 11.8h, 100m = 118h ou 5 dias
#(4090) Para 84 tokens(tokenmonster salva 35%) e 24000 vocab(Electra): 1000 = 3.5s, 10000 = 35s, 100000 = 5.9m, 1m = 59min, 10m = 9.8h, 100m = 98h ou 4 dias
#(4090) Para 84 tokens(tokenmonster salva 35%) e 16000 vocab (Electra): 1000 = 2.1s, 10000 = 21s, 100000 = 3.53m, 1m = 35.3min, 10m = 5.8h, 100m = 58h ou 2.4 dias

#com retnet. Confirmado que é O(n). Se lembrando que não existe mta vantagem além do custo crescer sequencialmente, já que o conteudo 
#(4090) Para 84 tokens(tokenmonster salva 35%) batch size 200 e 16000 vocab (Electra): 1000 = 2.1s, 10000 = 21s, 100000 = 3.53m, 1m = 35.3min, 10m = 5.8h, 100m = 58h ou 2.4 dias
#(4090) Para 168 tokens(tokenmonster salva 35%) batch size 100 e 16000 vocab (Electra): 1000 = 3.7s, 10000 = 37.5s,
#(4090) Para 1000 tokens(tokenmonster salva 35%) batch size 12 e 16000 vocab (Electra):            , 10000 = 4min32s,


#HIPOTÉTICO(Se conseguisse representar um texto com menos tokens):
#(4090) Para 24 tokens(tokenmonster salva 35%)(Electra): 1000 = , 10000 = , 100000 = 3.19m, 1m = 32min, 10m = 6.5h, 100m = 65h ou 2.7 dias
#HIPOTÉTICO(Se diminuisse o vocabulario para 1024 tokens diferentes)
#(4090) Para 84 tokens(tokenmonster salva 35%) e 1024 vocab: 1000 =1.6s , 10000 = 16.2s,

#losses:
#1m dataset, electra, tokenmonster: iteration:  7800 , total_loss:  26.277032788594564
#10m dataset, 84 tokens(tokenmonster salva 35%) e 16000 vocab (Electra) iteration:  49980 , total_loss:  20.8733185450236. 339min total


LIMIT_DATASET = 10_000
RANDOM_SEED = 42
NUM_TOKENIZER_TRAINING_ITEMS = 1_000_000  # I made this up, but it seems reasonable
# VOCAB_SIZE = 32_768  # from Cramming
# DEVICE_BATCH_SIZE = 100 # aprox 128, adjust to get near 100% gpu memory use
# MODEL_MAX_SEQ_LEN = 128  # from Cramming

VOCAB_SIZE = 16_000  # tokenmonster
# VOCAB_SIZE = 1_024  # tokenmonster
DEVICE_BATCH_SIZE = 20 # Token monster aguenta um batch size de (200-248)!! Geralmente melhora a qualidade do treino
MODEL_MAX_SEQ_LEN = 84 # token_monster


MASK_ID=4
PAD_ID=0
gradient_accumulation_steps = 2048 // DEVICE_BATCH_SIZE  # roughly based on Cramming
batch_size = DEVICE_BATCH_SIZE * gradient_accumulation_steps
print('batch_size: ',batch_size)
RUN_DIR = Path("data") / f"run_{time.strftime('%Y%m%d-%H%M%S')}"
CHECKPOINT_DIR = RUN_DIR / "training_checkpoints"
MODEL_DIR = RUN_DIR / "model"
TOKENIZER_PATH = RUN_DIR / "tokenizer.json"
TRAINER_HISTORY_PATH = RUN_DIR / "trainer_history.json"

RUN_DIR.mkdir(exist_ok=True, parents=True)

dataset = datasets.load_dataset(
    "sradc/chunked-shuffled-wikipedia20220301en-bookcorpusopen",
    split=f"train[:{LIMIT_DATASET}]" if LIMIT_DATASET else "train",
    revision="0e6fada2dd43136e4a3f637da41de2e596aee674",
)
print('loaded dataset!!')

from process_tokenizer import get_tokenizer
# tokenizer,tokenized_dataset,norm,vocab=get_tokenizer(dataset,NUM_TOKENIZER_TRAINING_ITEMS,VOCAB_SIZE,TOKENIZER_PATH,'clm',MODEL_MAX_SEQ_LEN+2)
tokenized_dataset,norm,vocab=get_tokenizer(dataset,NUM_TOKENIZER_TRAINING_ITEMS,VOCAB_SIZE,TOKENIZER_PATH,'mlm',MODEL_MAX_SEQ_LEN+2)
# tokenized_dataset,norm,vocab=get_tokenizer(dataset,NUM_TOKENIZER_TRAINING_ITEMS,VOCAB_SIZE,TOKENIZER_PATH,'smlm',MODEL_MAX_SEQ_LEN+2)
MASK_ID=vocab.tokenize("[MASK]")[0]
N_DOMAINS=3


batch_size:  2040


Resolving data files: 100%|██████████| 53/53 [00:00<00:00, 73.57it/s]


loaded dataset!!


In [3]:
import string
from fastbm25 import fastbm25
corpus = [text for text in dataset['text']]
corpus = [doc.translate(str.maketrans('', '', string.punctuation)).replace('\n',"").lower().strip().split() for doc in corpus]
bm25 = fastbm25(corpus)
is_domain=False
SEQ_LEN=MODEL_MAX_SEQ_LEN

In [4]:
#given a query string, and a list of subwords, and embeddings for each subwords, return a list of embeddings for each word
def get_word2subword(query):
    tokens  = vocab.tokenize(query)
    # tokens_str = [vocab.decode([int(i)]) for i in tokens]
    #for i in range(len(tokens_str)):
    #    print('token',i,':',tokens_str[i])
    sentence_token_id = 0
    decoder = vocab.decoder()
    word2subword_map = [{p:[]} for p in query.split()]

    temp_word = ""
    temp_subwords = []

    for id, token_id in enumerate(tokens):
      token = decoder.decode(tokens[id]).strip()
      subwords = token.split()

      for sub_id, sub in enumerate(subwords):
        word_map = word2subword_map[sentence_token_id]

        if sub in word_map:
          word_map[sub] = [id]
          sentence_token_id += 1
        else:
          temp_word += sub
          temp_subwords.append(id)

        if temp_word in word_map:
            word_map[temp_word] = temp_subwords
            temp_word = ""
            temp_subwords = []
            sentence_token_id += 1

    return word2subword_map 

#join subword embeddings
def get_words_embs(x,query):
    #query = vocab.decode([int(i) for i in query_ids[0]])

    # query=' '.join(query.translate(str.maketrans('', '', string.punctuation)).replace('\n',"").lower().split())
    query = ' '.join(query.lower().split())

    query=query.strip()
    words_embs=[]
    # print('query: ',query)
    word2subword_map = get_word2subword(query)
    #print('x.shape: ',x.shape)
    #print('word2subword_map: ',word2subword_map)
    for i in range(len(word2subword_map)):
        word_map = word2subword_map[i]
        word = list(word_map.keys())[0]
        subword_ids = []
        for id in word_map[word]:
            # if id < MODEL_MAX_SEQ_LEN:
            if id < SEQ_LEN:
                subword_ids.append(id)
        subword_embs = x[:,subword_ids,:]
        word_emb = subword_embs.mean(dim=1)
        words_embs.append(word_emb)
    words_embs = torch.stack(words_embs,dim=1)
    #print('words_embs.shape: ',words_embs.shape)
    return words_embs

def search_similars(query,weights,n=10,domain=0):
    #print('query: ',query)
    tokenized_query =query.translate(str.maketrans('', '', string.punctuation)).replace('\n',"").lower().split()
    #print('tokenized_query: ',tokenized_query)
    #print(len(tokenized_query))
    if is_domain:
        result = domain_bm25[domain].top_k_sentence_weighted(tokenized_query,weights,k=n)
    else:
        result = bm25.top_k_sentence_weighted(tokenized_query,weights,k=n)
        # result = bm25.top_k_sentence(tokenized_query,k=n)

    sorted_scores=[r[1] for r in result]
    top_n = [dataset['text'][i] for i in sorted_scores[1:n+1]]
    return top_n
    
def trunc_pad(similar_docs,n=4):
    trunc_pad_similar_docs=[]
    c=0
    for doc in similar_docs:
        if c==0:
            doc='[RET]'+doc
        doc=vocab.tokenize(doc)[:SEQ_LEN]
        doc=list(doc)
        for j in range(SEQ_LEN-len(doc)):
            doc.append(PAD_ID)
        trunc_pad_similar_docs.append(doc)
        c+=1
    #caso não tenha docs suficientes
    for i in range(n-len(similar_docs)):
        trunc_pad_similar_docs.append([PAD_ID]*SEQ_LEN)

    tok = vocab.tokenize('[RET]')
    trunc_pad_similar_docs[-1][-len(tok):]=tok
    similar_docs=trunc_pad_similar_docs
    return similar_docs


In [5]:
# hidden layer extractor class, for magically adding adapter to language model to be pretrained
class HiddenLayerExtractor(nn.Module):
    def __init__(self, net, layer = -2):
        super().__init__()
        self.net = net
        self.layer = layer

        self.hidden = None
        self.hook_registered = False

    def _find_layer(self):
        if type(self.layer) == str:
            modules = dict([*self.net.named_modules()])
            return modules.get(self.layer, None)
        elif type(self.layer) == int:
            children = [*self.net.children()]
            return children[self.layer]
        return None

    def _hook(self, _, __, output):
        self.hidden = output

    def _register_hook(self):
        layer = self._find_layer()
        assert layer is not None, f'hidden layer ({self.layer}) not found'
        handle = layer.register_forward_hook(self._hook)
        self.hook_registered = True

    def forward(self, x):
        if self.layer == -1:
            return self.net(x)

        if not self.hook_registered:
            self._register_hook()

        _ = self.net(x)
        hidden = self.hidden
        self.hidden = None
        assert hidden is not None, f'hidden layer {self.layer} never emitted an output'
        return hidden

In [6]:
from x_transformers import TransformerWrapper, Decoder,Encoder

RETRIEVAL_NUM=5

#encoder decoder
colt5_dict={
            'encoder':{'is_colt5_encoder':True,'light_ff_mult':0.5,'heavy_ff_mult':4,'num_heavy_tokens':300,
                   'light_dim_head':64,'light_heads':8,'light_window_size':128,
                   'heavy_dim_head':64,'heavy_heads':8,'num_heavy_tokens_q':300,
                   'num_heavy_tokens_kv':300},
            }

# model = TransformerWrapper(
model = TransformerWrapper(
    num_tokens = VOCAB_SIZE+8,
    max_seq_len = MODEL_MAX_SEQ_LEN*(RETRIEVAL_NUM+1),
    attn_layers = Encoder(
        dim = int(768),
        depth = 12,
        heads =12,
        attn_flash = True,
        colt5_dict=colt5_dict['encoder'],
        # rel_pos_bias = True
    ),
    emb_dim=128,
)
feature_extractor = HiddenLayerExtractor(model, layer=-2)
query_weights_layer = torch.nn.Linear(768, 1)

In [7]:
# input_ids are the indices corresponding to each token in the sentence.
# attention_mask indicates whether a token should be attended to or not.
# token_type_ids identifies which sequence a token belongs to when there is more than one sequence

train_dataloader = DataLoader(
        tokenized_dataset, shuffle=True, batch_size=DEVICE_BATCH_SIZE
    )
from utils import get_optimizer_scheduler
optimizer,lr_scheduler,max_train_steps = get_optimizer_scheduler(model,train_dataloader,gradient_accumulation_steps,learning_rate=5e-5,weight_decay=0, num_warmup_steps=0, max_train_steps=None,lr_scheduler_type='linear',num_train_epochs=1)

accelerator = Accelerator()
# Prepare everything with our `accelerator`.
model, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
    model, optimizer, train_dataloader, lr_scheduler
)
feature_extractor=feature_extractor.to(accelerator.device)
query_weights_layer=query_weights_layer.to(accelerator.device)

In [8]:
import numpy as np

def add_retrival_docs(output_words,words_embs,query_weights,query_ids):
    similar_docs= search_similars(output_words,query_weights,RETRIEVAL_NUM)
    #trunc and pad
    similar_docs=trunc_pad(similar_docs,RETRIEVAL_NUM)
    similar_docs = np.asarray(similar_docs,dtype=np.int32)
    similar_docs = torch.from_numpy(similar_docs).to(words_embs.device)
    similar_docs = similar_docs.reshape((1,similar_docs.shape[0]*similar_docs.shape[1]))
    input_ids = torch.cat((similar_docs[0],query_ids[0]),dim=0)
    input_ids = input_ids.flatten().unsqueeze(0)
    return input_ids
        

In [9]:
# Only show the progress bar once on each machine.
# progress_bar = tqdm(range(max_train_steps), disable=not accelerator.is_local_main_process)
completed_steps = 0
starting_epoch = 0

# update the progress_bar if load from checkpoint
total_loss=0
# progress_bar.update(completed_steps)
steps_log=30
count_amostra=0
num_train_epochs=1
for epoch in range(starting_epoch, num_train_epochs):
    
    for step, batch in enumerate(train_dataloader):
        # model.eval()
        # feature_extractor.eval()
        ### STAGE 1: Retrieval ###
        query_ids=batch['input_ids']
        # print('query_ids.shape: ',query_ids.shape)

        #get subwords embeddings and convert to word embeddings for term weighting later
        subwords_embs = feature_extractor(query_ids)
        # print('subwords_embs.shape: ',subwords_embs.shape)
        #call to_logits layer to replace masked tokens with predictions
        output_logits = model.to_logits(subwords_embs)
        output_ids = torch.argmax(output_logits,dim=-1)
        output_words = vocab.decode([int(i) for i in output_ids[0]])
        # print('output_words: ',output_words)
        #get words embeddings to calculate weights
        words_embs = get_words_embs(subwords_embs,output_words)
        # print('words_embs.shape: ',words_embs.shape)

        #search similar docs with weights computed from word embs, and add to input_ids
        query_weights = query_weights_layer(words_embs)[0]
        query_weights = query_weights.flatten()
        query_weights = torch.sigmoid(query_weights)
        input_ids=add_retrival_docs(output_words,words_embs,query_weights,query_ids)
        # print('inputs.shape: ',input_ids.shape)

        model.train()
        ### STAGE 2: Training ###
        logits=model(input_ids) 
        logits=logits[:,RETRIEVAL_NUM*MODEL_MAX_SEQ_LEN:,:]
        input_ids=input_ids[:,RETRIEVAL_NUM*MODEL_MAX_SEQ_LEN:]
        # print('logits.shape: ',logits.shape)
        # print('input_ids.shape: ',input_ids.shape)

        loss = F.cross_entropy(logits.transpose(1, 2),input_ids,ignore_index = PAD_ID)
        count_amostra+=int(len(batch['input_ids']))
        # print(loss)
        total_loss += loss.detach().float().cpu().numpy().item()
        accelerator.backward(loss)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()   
        
        if step%steps_log==0:
            print('iteration: ',step,', total_loss: ',total_loss/steps_log)
            print('count_amostra:',count_amostra)
            total_loss=0
    # Checks if the accelerator has performed an optimization step behind the scenes
    if accelerator.sync_gradients:
        # progress_bar.update(1)
        completed_steps += 1

    if completed_steps >= max_train_steps:
        break

    model.eval()
    losses = []


iteration:  0 , total_loss:  0.32674589157104494
count_amostra: 20
iteration:  30 , total_loss:  8.696722904841105
count_amostra: 620
iteration:  60 , total_loss:  7.973774194717407
count_amostra: 1220


## Evaluate

In [10]:
model_extractor = HiddenLayerExtractor(model,-2)

In [11]:
import transformers
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
import os
from torch.utils.data import  DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer

class Classifier(torch.nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # self.l1 = model._modules['model']
        self.l1 = model_extractor
        self.l2 = torch.nn.Dropout(0.3)
        self.fl = torch.nn.Flatten()
        self.l3 = torch.nn.Linear(768*(MODEL_MAX_SEQ_LEN-2), 3)
    
    def forward(self, ids):
        output= self.l1(ids)
        # print('l1:',output.shape)
        output = self.l2(output)
        # print('l2:',output.shape)
        output = self.fl(output)
        # print('fl:',output.shape)
        output = self.l3(output)
        # print('l3:',output.shape)
        return output

classifier_model = Classifier()
classifier_model=classifier_model.to('cuda')
optimizer = torch.optim.Adam(params =  classifier_model.parameters(), lr=2e-5,weight_decay=0.01)

In [12]:
task='mrpc'
dataset = load_dataset('glue', task, split='train')

In [13]:
import numpy as np
class TokenizedDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.ls=[]

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        s=norm.normalize_str(self.dataset["sentence1"][i])+'[SEP]'+norm.normalize_str(self.dataset["sentence2"][i])
        tokens = vocab.tokenize(s).tolist()
        
        self.ls.append(i)
        
        #trucate
        tokens=tokens[:MODEL_MAX_SEQ_LEN - 2]
        l=len(tokens)
        for j in range(l,MODEL_MAX_SEQ_LEN - 2):
            tokens.append(0)
        tokens=torch.Tensor(tokens).long()

        label=self.dataset["label"][i]
        
        d={'input_ids':tokens,'label':label}
        return d



from torch.utils.data import DataLoader

tokenized_dataset = TokenizedDataset(dataset)
# train_dataloader=CustomDataloader(tokenized_dataset)
train_dataloader = DataLoader(
        tokenized_dataset, batch_size=DEVICE_BATCH_SIZE
    )

In [14]:
import numpy as np
from torch.autograd import Variable

classifier_model.train()

def loss_fn(outputs, targets):
    loss=torch.nn.CrossEntropyLoss()(outputs, targets)
    return loss

In [15]:
#FINETUNING

epochs=5
running_loss=0
for i in range(epochs):
    for _,data in enumerate(train_dataloader):
        ids = data['input_ids'].to('cuda', dtype = torch.long)
        targets = data['label'].to('cuda', dtype = torch.long)

        outputs = classifier_model(ids)
        
        loss = loss_fn(outputs,targets)
        # loss=outputs.loss

        optimizer.zero_grad()
        running_loss+=loss.item()
        if _%50==0:
            print(f'Epoch: {i}, Loss:  {running_loss/50}')
            running_loss=0
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Epoch: 0, Loss:  0.02589289903640747
Epoch: 0, Loss:  0.7359633594751358
Epoch: 0, Loss:  0.6966659402847291
Epoch: 0, Loss:  0.6798816281557083
Epoch: 1, Loss:  0.4578602337837219
Epoch: 1, Loss:  0.7293077337741852
Epoch: 1, Loss:  0.6903081750869751
Epoch: 1, Loss:  0.6618105518817902
Epoch: 2, Loss:  0.4600534611940384
Epoch: 2, Loss:  0.677624973654747
Epoch: 2, Loss:  0.6836879336833954
Epoch: 2, Loss:  0.6674785310029984
Epoch: 3, Loss:  0.44524687469005586
Epoch: 3, Loss:  0.6862362623214722
Epoch: 3, Loss:  0.6841507893800736
Epoch: 3, Loss:  0.6469327342510224
Epoch: 4, Loss:  0.45256554782390596
Epoch: 4, Loss:  0.7167031174898147
Epoch: 4, Loss:  0.6726771318912506
Epoch: 4, Loss:  0.6612649184465408


In [16]:
#EVALUATE
dataset = load_dataset('glue', task, split='validation')
tokenized_dataset = TokenizedDataset(dataset)

eval_dataloader = DataLoader(
        tokenized_dataset, batch_size=DEVICE_BATCH_SIZE
    )

In [17]:
model.eval()
predictions=[]
labels=[]
c=0
for _,data in enumerate(eval_dataloader):
    ids = data['input_ids'].to('cuda', dtype = torch.long)
    targets = data['label'].to('cuda', dtype = torch.long)
    
    outputs = classifier_model(ids)
    
    outputs=torch.argmax(outputs, dim=1).cpu()
    targets=targets.cpu()
    
    outputs=[int(o.item()) for o in outputs]
    targets=[int(o.item()) for o in targets]

    predictions+=outputs
    labels+=targets

In [18]:
from sklearn.metrics import classification_report
print(classification_report(labels, predictions))

              precision    recall  f1-score   support

           0       0.25      0.01      0.02       129
           1       0.68      0.99      0.81       279

    accuracy                           0.68       408
   macro avg       0.47      0.50      0.41       408
weighted avg       0.55      0.68      0.56       408



In [19]:
#Bert 3 layers  10000 dataset
#               precision    recall  f1-score   support

#            0       0.61      0.09      0.15       129
#            1       0.70      0.97      0.81       279

#     accuracy                           0.69       408
#    macro avg       0.65      0.53      0.48       408
# weighted avg       0.67      0.69      0.60       408